In [1]:
from lib.dataset.mesh_util import *
#from lib.smplx.body_models import SMPLX
from lib.dataset.TestDataset import TestDataset
from lib.common.config import cfg
from lib.pymafx.core import path_config
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
SMPLX_object = SMPLX()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

Type of each parameter in 'dataset_param':

    <class 'str'>
    <class 'NoneType'>
    <class 'bool'>
    <class 'str'>
    <class 'int'>
    <class 'bool'>

In [4]:
dataset_param = {
        "image_dir": './examples/testimg/',
        "seg_dir": None,
        "use_seg": True,    # w/ or w/o segmentation
        "hps_type": 'pymafx' ,   # pymafx/pixie
        "vol_res": cfg.vol_res,
        "single": False, #args.multi,
    }


In [5]:
path_config.pymafx_data_dir = "./data/HPS/pymafx_data"

os.path.exists(path_config.pymafx_data_dir)

True

In [6]:
param_npz = np.load('./00006_0.npz' , allow_pickle=True)
print(type(param_npz))
for item in param_npz:
    print (item)

<class 'numpy.lib.npyio.NpzFile'>
global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [7]:
param = {key: torch.as_tensor(param_npz[key]) for key in param_npz.keys()}

In [8]:
for item in param_npz:
    print (item)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [9]:

for key in param.keys():
    print(key)

global_orient
body_pose
left_hand_pose
right_hand_pose
jaw_pose
leye_pose
reye_pose
betas
expression
transl


In [10]:
from apps.infer import view_dict_items

In [11]:
view_dict_items(param)

global_orient: torch.Size([1, 3])
body_pose: torch.Size([21, 3])
left_hand_pose: torch.Size([15, 3])
right_hand_pose: torch.Size([15, 3])
jaw_pose: torch.Size([1, 3])
leye_pose: torch.Size([1, 3])
reye_pose: torch.Size([1, 3])
betas: torch.Size([1, 10])
expression: torch.Size([1, 10])
transl: torch.Size([1, 3])


In [12]:
#SMPLX_object = SMPLX('00006_0.npz')

In [13]:
data = np.load('./results/econ/obj/000023.jpg_0.npz',allow_pickle=True)
#print(data)
#data= data.tolist()
for item in data:
    print(item,': ',data[item].shape)

print(data['scale'])

global_orient :  (1, 3)
body_pose :  (21, 3)
left_hand_pose :  (15, 3)
right_hand_pose :  (15, 3)
jaw_pose :  (1, 3)
leye_pose :  (1, 3)
reye_pose :  (1, 3)
betas :  (1, 10)
expression :  (1, 10)
transl :  (1, 3)


KeyError: 'scale is not a file in the archive'

In [23]:
npyfile = './pymafx_results/econ/obj/OIP_smpl_00.npy'
NpzFile = './00006_0.npz'

In [24]:
data = np.load(npyfile,allow_pickle=True)
data = data.tolist()

In [28]:
for item in data :
    print(item)

betas
body_pose
global_orient
transl
expression
jaw_pose
left_hand_pose
right_hand_pose
scale


In [29]:
smpl_mesh,smpl_joint = load_fit_body(fitted_path=npyfile,scale=1)

<class 'numpy.ndarray'>


KeyError: 'leye_pose'

In [ ]:
def axis_angle_to_rotation_matrix(axis_angle: Tensor) -> Tensor:
    r"""Convert 3d vector of axis-angle rotation to 3x3 rotation matrix.

    Args:
        axis_angle: tensor of 3d vector of axis-angle rotations in radians with shape :math:`(N, 3)`.

    Returns:
        tensor of rotation matrices of shape :math:`(N, 3, 3)`.

    Example:
        >>> input = tensor([[0., 0., 0.]])
        >>> axis_angle_to_rotation_matrix(input)
        tensor([[[1., 0., 0.],
                 [0., 1., 0.],
                 [0., 0., 1.]]])

        >>> input = tensor([[1.5708, 0., 0.]])
        >>> axis_angle_to_rotation_matrix(input)
        tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00],
                 [ 0.0000e+00, -3.6200e-06, -1.0000e+00],
                 [ 0.0000e+00,  1.0000e+00, -3.6200e-06]]])
    """
    if not isinstance(axis_angle, Tensor):
        raise TypeError(f"Input type is not a Tensor. Got {type(axis_angle)}")

    if not axis_angle.shape[-1] == 3:
        raise ValueError(f"Input size must be a (*, 3) tensor. Got {axis_angle.shape}")

    def _compute_rotation_matrix(axis_angle: Tensor, theta2: Tensor, eps: float = 1e-6) -> Tensor:
        # We want to be careful to only evaluate the square root if the
        # norm of the axis_angle vector is greater than zero. Otherwise
        # we get a division by zero.
        k_one = 1.0
        theta = torch.sqrt(theta2)
        wxyz = axis_angle / (theta + eps)
        wx, wy, wz = torch.chunk(wxyz, 3, dim=1)
        cos_theta = cos(theta)
        sin_theta = sin(theta)

        r00 = cos_theta + wx * wx * (k_one - cos_theta)
        r10 = wz * sin_theta + wx * wy * (k_one - cos_theta)
        r20 = -wy * sin_theta + wx * wz * (k_one - cos_theta)
        r01 = wx * wy * (k_one - cos_theta) - wz * sin_theta
        r11 = cos_theta + wy * wy * (k_one - cos_theta)
        r21 = wx * sin_theta + wy * wz * (k_one - cos_theta)
        r02 = wy * sin_theta + wx * wz * (k_one - cos_theta)
        r12 = -wx * sin_theta + wy * wz * (k_one - cos_theta)
        r22 = cos_theta + wz * wz * (k_one - cos_theta)
        rotation_matrix = concatenate([r00, r01, r02, r10, r11, r12, r20, r21, r22], dim=1)
        return rotation_matrix.view(-1, 3, 3)

    def _compute_rotation_matrix_taylor(axis_angle: Tensor) -> Tensor:
        rx, ry, rz = torch.chunk(axis_angle, 3, dim=1)
        k_one = torch.ones_like(rx)
        rotation_matrix = concatenate([k_one, -rz, ry, rz, k_one, -rx, -ry, rx, k_one], dim=1)
        return rotation_matrix.view(-1, 3, 3)

    # stolen from ceres/rotation.h

    _axis_angle = torch.unsqueeze(axis_angle, dim=1)
    theta2 = torch.matmul(_axis_angle, _axis_angle.transpose(1, 2))
    theta2 = torch.squeeze(theta2, dim=1)

    # compute rotation matrices
    rotation_matrix_normal = _compute_rotation_matrix(axis_angle, theta2)
    rotation_matrix_taylor = _compute_rotation_matrix_taylor(axis_angle)

    # create mask to handle both cases
    eps = 1e-6
    mask = (theta2 > eps).view(-1, 1, 1).to(theta2.device)
    mask_pos = (mask).type_as(theta2)
    mask_neg = (~mask).type_as(theta2)

    # create output pose matrix with masked values
    rotation_matrix = mask_pos * rotation_matrix_normal + mask_neg * rotation_matrix_taylor
    return rotation_matrix  # Nx3x3

